In [21]:
import ee
import numpy as np
import folium
import geopandas as gpd
import json
from util import shp_to_ee_fmt,add_ee_layer,get_whole_region
import time 
from yearly_bar_plot_tree_cover import pixel_stats_dynamic_world,plot_multiple_bar_from_dict,pixel_stats_world_cover,pixel_stats_global_forest_watch
from constants import * 
import matplotlib.pyplot as plt
try:     
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()